In [ ]:
import os, re
from pathlib import Path

def estandarizar_nombres_experimento(ruta_directorio, numero_exp, eliminar_fecha=True, simulacro=True):
    """
    Renombra archivos actualizando el número de experimento y eliminando fechas.
    
    Args:
        ruta_directorio (str): Ruta de la carpeta a procesar.
        numero_exp (int/str): El nuevo número 'i' para 'exp_i'.
        eliminar_fecha (bool): Si es True, borra el timestamp final (ej: _20251018_215241).
        simulacro (bool): Si es True, solo imprime qué pasaría sin tocar nada.
    """
    
    ruta = Path(ruta_directorio)
    
    # Validar que la carpeta existe
    if not ruta.exists():
        print(f"❌ Error: La ruta '{ruta}' no existe.")
        return

    print(f"--- Procesando: {ruta.name} (Simulacro: {simulacro}) ---")

    # 1. Definimos los patrones (Regex)
    # Busca 'exp_' seguido de uno o más dígitos (\d+)
    patron_exp = re.compile(r'exp_\d+')
    
    # Busca un guion bajo, 8 dígitos (fecha), guion bajo, 6 dígitos (hora)
    # Ajustado a lo que vi en tu imagen: _20251018_215241
    patron_fecha = re.compile(r'_\d{8}_\d{6}') 

    archivos_modificados = 0

    for archivo in ruta.iterdir():
        if archivo.is_file():
            nombre_viejo = archivo.name
            nombre_nuevo = nombre_viejo

            # A. Actualizar el número de experimento
            # Reemplaza lo que encuentre en el patrón por 'exp_{numero_exp}'
            nombre_nuevo = patron_exp.sub(f'exp_{numero_exp}', nombre_nuevo)

            # B. Eliminar la fecha si se solicita
            if eliminar_fecha:
                nombre_nuevo = patron_fecha.sub('', nombre_nuevo)

            # C. Ejecutar el cambio si el nombre es diferente
            if nombre_nuevo != nombre_viejo:
                if simulacro:
                    print(f"📝 [SIM] {nombre_viejo} \n       -> {nombre_nuevo}")
                else:
                    try:
                        archivo.rename(ruta / nombre_nuevo)
                        print(f"✅ [OK] ... -> {nombre_nuevo}")
                    except OSError as e:
                        print(f"❌ [Error] {nombre_viejo}: {e}")
                
                archivos_modificados += 1

    if archivos_modificados == 0:
        print("No se encontraron archivos que coincidieran con los patrones o que necesitaran cambios.")
    else:
        print(f"\nProceso completado. Archivos afectados: {archivos_modificados}")

In [ ]:
# 1. Definir la ruta
script_path = os.getcwd()
project_path = os.path.join(script_path, '..')
DATA_ID = 'dubai_houses'
EXPERIMENT_ID = 'experiment_2'
folder_path = os.path.join(project_path, 'results', EXPERIMENT_ID, DATA_ID)

# 2. Ejecutar primero en modo SIMULACRO (recomendado)
# Esto cambiará 'exp_X' a 'exp_2' y borrará la fecha.
estandarizar_nombres_experimento(
    ruta_directorio=folder_path, 
    numero_exp=2, 
    eliminar_fecha=True, 
    simulacro=True  # <--- Cambia a False cuando estés listo
)